In [5]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [6]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)

from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario, SequentialEvaluator)

from ema_workbench.em_framework.evaluators import perform_experiments, optimize
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, utilities

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [7]:
from model.dike_model_function import DikeNetwork  # @UnresolvedImport
from model.problem_formulation import get_model_for_problem_formulation

C:\Users\newbi\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [8]:
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [9]:
from ema_workbench import MultiprocessingEvaluator

with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(
        scenarios = 10,
        policies = 4,
        levers_sampling = 'mc' # Monte-Carlo
    )

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 10 scenarios * 4 policies * 1 model(s) = 40 experiments
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 24 cases completed
[MainProcess/INFO] 28 cases completed
[MainProcess/INFO] 32 cases completed
[MainProcess/INFO] 36 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [26]:
MAXIMIZE = ScalarOutcome.MAXIMIZE
MINIMIZE = ScalarOutcome.MINIMIZE

def my_func(data):
    mean = np.mean(data)
    iqr = sp.stats.iqr(data) + 1
    
    return mean * iqr
    

# These functions need to only return one value...

robustness_functions = [
    ScalarOutcome('Damage Score', function = my_func, kind = MINIMIZE,
                  variable_name=['A.1_Expected Annual Damage', 'A.2_Expected Annual Damage',
                                 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage']),    
    
    ScalarOutcome('Deaths Score', function = my_func, kind = MINIMIZE,
                  variable_name=['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths',
                                 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths']),
    
    ScalarOutcome('Dike Invest Score', function = my_func, kind = MINIMIZE,
                  variable_name=['A.1_Dike Investment Costs', 'A.2_Dike Investment Costs',
                                 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs']),
    
    ScalarOutcome('RfR Invest Score', variable_name=['RfR Total Costs'], function = my_func, kind = MINIMIZE),
    
    ScalarOutcome('Evac Score', variable_name=['Expected Evacuation Costs'], function = my_func, kind = MINIMIZE)
    
]

In [11]:
from ema_workbench.em_framework import sample_uncertainties

n_scenarios = 2
scenarios = sample_uncertainties(dike_model, n_scenarios)
nfe = int(2)

In [13]:
from ema_workbench import ema_logging
from ema_workbench.em_framework.optimization import (HyperVolume, 
                                                     EpsilonProgress)
from ema_workbench.em_framework.evaluators import BaseEvaluator

BaseEvaluator.reporting_frequency = 0.1
ema_logging.log_to_stderr(ema_logging.INFO)

epsilons = [0.05,]*len(robustness_functions)
convergence = [HyperVolume(minimum=[0,0,0], maximum=[1.1, 1.1, 1.1]),
              EpsilonProgress()]

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.robust_optimize(robustness_functions,
                                        scenarios = scenarios,
                                        nfe=nfe,
                                        epsilons=epsilons,
                                        convergence=convergence)

TypeError: object of type 'int' has no len()